Utility Functions
- NOTE: tensors are organized as follows: [BATCH_SIZE, CHANNELS, WIDTH, HEIGHT]

In [ ]:
import torch
import torch.nn.functional as func
import PIL.Image
import random
import json
import pathlib
import numpy as np
import matplotlib.pylab as pl
import matplotlib.gridspec as gridspec

from torchvision.transforms.functional_tensor import gaussian_blur
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tqdm import tqdm

# * loads an image and converts to a tensor
# *     default tensor shape: [BATCH_SIZE (1), CHANNELS (4), WIDTH (_size), HEIGHT (_size)]
def load_image_as_tensor(_path, _size, _resample=PIL.Image.Resampling.BICUBIC):
    img = PIL.Image.open(_path)
    img = img.resize((_size, _size), _resample)
    img = np.float32(img) / 255.0
    img[..., :3] *= img[..., 3:]
    return torch.from_numpy(img).permute(2, 0, 1)[None, ...]

# * given a tensor of default shape, visualize the first 4 channels as a RGBA image
def show_tensor_as_image(_tensor):
    img = to_rgb(_tensor).squeeze().permute(1, 2, 0)
    plt.axis('off')
    plt.imshow(img)
    plt.show()

# * takes the first 4 channels of a tensor of default shape and converts to a RGB image
def to_rgb(_x, _alpha='BLACK'):
    rgb, a = _x[:, :3], _x[:, 3:4]
    if _alpha == 'BLACK':
        return rgb
    elif _alpha == 'WHITE':
        return 1.-a + rgb

# * creates a circle mask centered at a position of a given radius
def circle_mask(_size, _radius, _pos):
    Y, X = np.ogrid[:_size, :_size]
    dist_from_center = np.sqrt((X - _pos[0])**2 + (Y-_pos[1])**2)
    mask = dist_from_center >= _radius
    return mask

# * shows a batch before and after a forward pass given two (2) tensors
def show_batch(_batch_size, _before, _after, _dpi=256):
    fig = plt.figure(figsize=(_batch_size, 2), dpi=_dpi)
    axarr = fig.subplots(nrows=2, ncols=_batch_size)
    gspec = gridspec.GridSpec(2, _batch_size)
    gspec.update(wspace=0.1, hspace=0) # set the spacing between axes.
    plt.clf()
    for i in range(_batch_size):
        img_i = _before[i].unsqueeze(0)
        img_rgb = to_rgb(img_i, _alpha='BLACK').squeeze().permute(1, 2, 0)
        img_rgb = (np.array(img_rgb)*255).astype(np.uint8)
        axarr[0, i] = plt.subplot(gspec[i])
        axarr[0, i].set_xticks([])
        axarr[0, i].set_yticks([])
        axarr[0, i].imshow(img_rgb, aspect='equal')
        axarr[0, i].set_title(str(i), fontsize=8)   
    for i in range(_batch_size):
        img_i = _after[i].unsqueeze(0)
        img_rgb = to_rgb(img_i, _alpha='BLACK').squeeze().permute(1, 2, 0)
        img_rgb = (np.array(img_rgb)*255).astype(np.uint8)
        axarr[1, i] = plt.subplot(gspec[i+_batch_size])
        axarr[1, i].set_xticks([])
        axarr[1, i].set_yticks([])
        axarr[1, i].imshow(img_rgb, aspect='equal') 
    plt.show()

# * blurs an image using gaussian blur
def unsharpen(_img):
    blur = gaussian_blur(_img, (5, 5), (1, 1))
    return _img + (_img - blur) * 2.0

# * find GPU available
clear_output()
!nvidia-smi -L

# * sets the device
# *     defaults to 'cuda'
_DEVICE_ = 'cuda' if torch.cuda.is_available() else 'cpu'
print ('cuda available? ', torch.cuda.is_available())
print ('device: ', _DEVICE_)

Load Pre-Made Seed

In [ ]:
_SEED_FILE_ = '_3_seeds_64.png'
_SIZE_ = 64

seed_img = load_image_as_tensor('..\\_seeds\\'+_SEED_FILE_,  _SIZE_)
print ('seed_img.shape: ', seed_img.shape)
show_tensor_as_image(to_rgb(seed_img))
show_tensor_as_image(to_rgb(seed_img, 'WHITE'))

Target Image to Train Model
- NOTE: seed _SIZE_ should equal _TARGET-SIZE_ + (2 * _PAD_)

In [ ]:
_TARGET_FILE_ = 'cowboy.png'
_TARGET_SIZE_ = 40
_PAD_ = 12

target_img = load_image_as_tensor('..\\_images\\'+_TARGET_FILE_, _TARGET_SIZE_)
target_img = torch.nn.functional.pad(target_img, (_PAD_, _PAD_, _PAD_, _PAD_), 'constant', 0)
print ('target_img.shape: ', target_img.shape)
show_tensor_as_image(target_img)

Perceptions:
- LAPLACIAN: isotropic nca model
- SOBEL_MAG: isotrpic nca variant which adds upon the 'laplacian' model by making use of the magnitude of the two directional sobel filters
- STEERABLE: angle-based steerable nca
- GRADIENT: gradient-based steerable nca

In [ ]:
SOBEL_KERN = torch.tensor([
    [-1., 0., 1.], 
    [-2., 1., 2.], 
    [-1., 0., 1.]])
LAP_KERN = torch.tensor([
    [1.,   2., 1.], 
    [2., -12., 2.], 
    [1.,   2., 1.]])
ID_KERN = torch.tensor([
    [0., 0., 0.], 
    [0., 1., 0.], 
    [0., 0., 0.]])

# * performs a convolution per filter per channel
def per_channel_conv(_x, _filters):
    batch_size, channels, height, width = _x.shape
    # * reshape x to make per-channel convolution possible + pad 1 on each side
    y = _x.reshape(batch_size*channels, 1, height, width)
    y = func.pad(y, (1, 1, 1, 1), 'circular')
    # send to current device
    _filters = _filters.to(_DEVICE_)
    y = y.to(_DEVICE_)
    # * perform per-channel convolutions
    y = func.conv2d(y, _filters[:, None])
    y = y.reshape(batch_size, -1, height, width)
    return y

# * only uses laplacian operator for local perception
def laplacian_perception(_x):
    # * add an extra dimention to account for batch size
    lap_conv = per_channel_conv(_x, LAP_KERN[None, :])
    # * concat perception w/ self (identity)
    y = torch.cat([_x, lap_conv], 1)
    return y

# * uses laplacian operator and sobel-magnitude (G) for local perception
def sobel_mag_perception(_x):
    # * add an extra dimention to account for batch size
    lap_conv = per_channel_conv(_x, LAP_KERN[None, :])
    # * compute sobel-magnitude (G)
    sobel_conv = per_channel_conv(_x, torch.stack([SOBEL_KERN, SOBEL_KERN.T]))
    gx, gy = sobel_conv[:, ::2], sobel_conv[:, 1::2]
    # * concat perceptions w/ self (identity)
    y = torch.cat([_x, lap_conv, (gx*gx+gy*gy+1e-8).sqrt()], 1)
    return y

"""
def steerable_perception(_x):
    # * separate states and angle channels
    states, angle = _x[:, :-1], _x[:, -1:]
    filters = torch.stack([SOBEL_KERN, SOBEL_KERN.T])
    grad = per_channel_conv(states, filters)
    gx, gy = grad[:, ::2], grad[:, 1::2]
    # * get cos and sin of angle channel and apply rotation to gx, gy
    c, s = angle.cos(), angle.sin()
    rot_grad = torch.cat([gx*c+gy*s, gy*c-gx*s], 1)
    lap_conv = per_channel_conv(_x, LAP_KERN[None, :])
    y = torch.cat([states, rot_grad, lap_conv], 1)

def gradient_perception(_x):
    filters = torch.stack([SOBEL_KERN, SOBEL_KERN.T])
    grad = per_channel_conv(_x, filters)
    grad, dir = grad[:, :-2], grad[:, -2:]
    dir = dir / dir.norm(dim=1, keepdim=True).clip(1.0)
    gx, gy = grad[:, ::2], grad[:, 1::2]
    c, s = dir[:, :1], dir[:, 1::2]
    rot_grad = torch.cat([gx*c+gy*s, gy*c-gx*s], 1)
    lap_conv = per_channel_conv(_x, LAP_KERN[None, :])
    y = torch.cat([_x, rot_grad, lap_conv], 1)
    return y
"""
    
perception = {
    'LAPLACIAN': laplacian_perception,
    'SOBEL_MAG': sobel_mag_perception,
    #'STEERABLE': steerable_perception,
    #'GRADIENT': gradient_perception    
}

def get_alive_mask(_x):
    return func.max_pool2d(_x[:, 3:4, :, :], kernel_size=3, stride=1, padding=1) > 0.1

In [ ]:
# * experiments and tests

a = torch.randn([1, 3, 3, 3]).to(_DEVICE_)
print ('a.shape: ', a.shape)
print ('type(a): ', type(a))
print ('a.dtype: ', a.dtype)
print ('a: ', a)

b = laplacian_perception(a)
print ('b.shape: ', b.shape)
print ('type(b): ', type(b))
print ('b.dtype: ', b.dtype)
print ('b: ', b)

Isotropic Neural Cellular Automata Model

In [ ]:
_CHANNELS_ = 16
_HIDDEN_ = 128
_MODEL_TYPE_ = 'LAPLACIAN'  #['LAPLACIAN', 'SOBEL_MAG', 'STEERABLE', 'GRADIENT']
_STOCHASTIC_UPDATE_RATE_ = 0.5
# * last state channel is angle and should be treated differently
_ANGLE_CHANNEL_ = 1 if _MODEL_TYPE_ == 'STEERABLE' else 0
_SCALAR_CHANNEL_ = _CHANNELS_ - _ANGLE_CHANNEL_

class ISO_NCA(torch.nn.Module):
    def __init__(self, _channels=_CHANNELS_, _hidden=_HIDDEN_, _device=_DEVICE_):
        super().__init__()
        self.device = _device
        # * determine number of perceived channels
        perception_channels = perception[_MODEL_TYPE_](torch.zeros([1, _channels, 8, 8]).to(_device)).shape[1]
        
        # * determine hidden channels (equalize the parameter count btwn model types)
        hidden_channels = 8*1024 // (perception_channels+_channels)
        hidden_channels = (_hidden+31) // 32*32
        
        print ('perception channels: ', perception_channels)
        print ('hidden channels: ', hidden_channels)
        
        # * model layers
        self.conv1 = torch.nn.Conv2d(perception_channels, hidden_channels, 1)
        self.conv2 = torch.nn.Conv2d(hidden_channels, _channels, 1, bias=False)
        with torch.no_grad():
            self.conv2.weight.data.zero_()
        
        # * send to device
        self.to(_device)
        
    def forward(self, _x):
        # * get alive mask
        alive_mask = get_alive_mask(_x).to(self.device)
        
        # * perception step
        _x = _x.to(self.device)
        p = perception[_MODEL_TYPE_](_x)
        
        # * update step
        p = self.conv2(torch.relu(self.conv1(p)))
        
        # * create stochastic update mask
        stochastic_mask = (torch.rand(_x[:, :1, :, :].shape) <= _STOCHASTIC_UPDATE_RATE_).to(self.device, torch.float32)
        
        # * perform update
        _x = _x + p * stochastic_mask
        if _SCALAR_CHANNEL_ == _CHANNELS_:
            _x = _x * alive_mask
        else:
            _x = torch.cat([_x[:, :_SCALAR_CHANNEL_]*alive_mask, _x[:, _SCALAR_CHANNEL_:] % (np.pi*2.0)], 1)
        return _x

# * print model parameter count
param_n = sum(p.numel() for p in ISO_NCA().parameters())
print('ISO-NCA param count:', param_n)

Loss Functions:
- PIXEL-WISE: L2-norm loss
- INVARIANT: rotation-invariant loss

In [ ]:
_LOSS_FUNC_ = 'PIXEL_WISE' #['PIXEL_WISE', 'INVARIANT']

# r = torch.linspace(0.5/_SIZE_, 1, _SIZE_//2.0)[:, None]
# a = torch.range(0, _SIZE_*np.pi)/(_SIZE_/2)
# polar_xy = torch.stack([r*a.cos(), r*a.sin()], -1)[None, :]
# polar_target = func.grid_sample(unsharpen(target_img[None, ...]), polar_xy)

# x = torch.linspace(-1, 1, _SIZE_)
# y, x = torch.meshgrid(x, x)
# xy_grid = torch.stack([x, y], -1)
# fft_target = torch.fft.rfft(polar_target).conj()
# polar_target_sqnorm = polar_target.square().sum(-1, keepdim=True)

def pixel_wise_loss_func(_x, _target, _scale=1e3, _dims=[]):
    return _scale * torch.mean(torch.square(_x[:, :4] - _target), _dims)

# def invariant_losses_func(_x):
#     img = unsharpen(_x)
#     polar_img = func.grid_sample(img, polar_xy.repeat(len(img), 1, 1, 1), mode='bicubic')
#     x = torch.fft.rfft(polar_img)
#     xy = torch.fft.irfft(x*fft_target)
#     xx = polar_img.square().sum(-1, keepdim=True)
#     yy = polar_target_sqnorm
#     diff = xx+yy-2.0*xy
#     return diff.mean([1, 2])

def invariant_loss_func(_x):
    raise NotImplementedError
    # return invariant_losses_func(_x).min(-1)[0].mean()

loss_func = {
    'PIXEL_WISE': pixel_wise_loss_func,
    'INVARIANT': invariant_loss_func
}

Create ISO-NCA Model

In [ ]:
_NAME_ = 'laplacian_cowboy'
_POOL_SIZE_ = 256
_BATCH_SIZE_ = 8
_LOWER_LR_ = 1e-5
_UPPER_LR_ = 1e-3

# * create model / optimizer / lr-scheduler
model = ISO_NCA()
opt = torch.optim.Adam(model.parameters(), _UPPER_LR_)
lr_sched = torch.optim.lr_scheduler.CyclicLR(opt, _LOWER_LR_, _UPPER_LR_, step_size_up=2000, mode='triangular2', cycle_momentum=False)

# * create target batch
target_batch = target_img.clone().repeat(_BATCH_SIZE_, 1, 1, 1).to(_DEVICE_)

# * create seed pool
seed = torch.cat([seed_img, torch.zeros([1, _CHANNELS_-4, _SIZE_, _SIZE_])], 1).to(_DEVICE_)
with torch.no_grad():
    pool = seed.clone().repeat(_POOL_SIZE_, 1, 1, 1)

Load ISO-NCA Model Checkpoint

In [ ]:
load_model = True
load_from = 'checkpoints\\laplacian_cowboy_cp4000'

def load_model_checkpoint():
    if not load_model: 
        return

    # * open params json file
    params = {}
    with open(load_from + '_params.json', 'r') as openfile:
        params = json.load(openfile)
    
    # * load in params
    _DEVICE_ = params['_DEVICE_']
    _SEED_FILE_ = params['_SEED_FILE_']
    _SIZE_ = params['_SIZE_']
    _NAME_ = params['_NAME_']
    _MODEL_TYPE_ = params['_MODEL_TYPE_']
    _POOL_SIZE_ = params['_POOL_SIZE_']
    _TARGET_FILE_ = params['_TARGET_FILE_']
    _TARGET_SIZE_ = params['_TARGET_SIZE_']
    _PAD_ = params['_PAD_']
    _BATCH_SIZE_ = params['_BATCH_SIZE_']
    _LOWER_LR_ = params['_LOWER_LR_']
    _UPPER_LR_ = params['_UPPER_LR_']
    _LOSS_FUNC_ = params['_LOSS_FUNC_']
    
    # * load state dictionary
    model.load_state_dict(torch.load(load_from + '.pt', map_location=_DEVICE_))   
    model.train()

    # * load seed
    seed_img = load_image_as_tensor('..\\_seeds\\'+_SEED_FILE_, _SIZE_)
    seed = torch.cat([seed_img, torch.zeros([1, _CHANNELS_-4, _SIZE_, _SIZE_])], 1).to(_DEVICE_)
    with torch.no_grad():
        pool = seed.clone().repeat(_POOL_SIZE_, 1, 1, 1)
        
    # * load target
    target_img = load_image_as_tensor('..\\_images\\'+_TARGET_FILE_, _TARGET_SIZE_)
    target_img = torch.nn.functional.pad(target_img, (_PAD_, _PAD_, _PAD_, _PAD_), 'constant', 0)
    target_batch = target_img.clone().repeat(_BATCH_SIZE_, 1, 1, 1).to(_DEVICE_)
    
    # * setup loss function etc.
    opt = torch.optim.Adam(model.parameters(), _UPPER_LR_)
    lr_sched = torch.optim.lr_scheduler.CyclicLR(opt, _LOWER_LR_, _UPPER_LR_, step_size_up=2000, mode='triangular2', cycle_momentum=False)
    
    print ('model loaded in successfully')

load_model_checkpoint()

Training Loop

In [ ]:
_SKIP_TRAIN_ = True
_EPOCHS_ = 10000
_NUM_DAMG_ = 1
_DAMG_RATE_ = 3
_INFO_RATE_ = 20
_SAVE_RATE_ = 1000

# * save model method
def save_model(_dir, _model, _name):
    model_path = pathlib.Path(_dir)
    model_path.mkdir(parents=True, exist_ok=True)
    torch.save(_model.state_dict(), _dir + '\\' + _name + '.pt')
    
    # * save model parameters
    dict = {
        # seed info
        '_SEED_FILE_': _SEED_FILE_,
        '_SIZE_': _SIZE_,
        # target image info
        '_TARGET_FILE_': _TARGET_FILE_,
        '_TARGET_SIZE_': _TARGET_SIZE_,
        '_PAD_': _PAD_,
        # model info
        '_CHANNELS_': _CHANNELS_,
        '_HIDDEN_': _HIDDEN_,
        '_MODEL_TYPE_': _MODEL_TYPE_,
        '_STOCHASTIC_UPDATE_RATE_': _STOCHASTIC_UPDATE_RATE_,
        '_ANGLE_CHANNEL_': _ANGLE_CHANNEL_,
        '_SCALAR_CHANNEL_': _SCALAR_CHANNEL_,
        # loss/lr info
        '_LOSS_FUNC_': _LOSS_FUNC_,
        '_LOWER_LR_': _LOWER_LR_,
        '_UPPER_LR_': _UPPER_LR_,
        # training info
        '_DEVICE_': _DEVICE_,
        '_NAME_': _NAME_,
        '_EPOCHS_': _EPOCHS_,
        '_POOL_SIZE_': _POOL_SIZE_,
        '_BATCH_SIZE_': _BATCH_SIZE_,
        '_NUM_DAMG_': _NUM_DAMG_,
        # training rate info
        '_DAMG_RATE_': _DAMG_RATE_,
        '_INFO_RATE_': _INFO_RATE_,
        '_SAVE_RATE_': _SAVE_RATE_,
    }
    json_object = json.dumps(dict, indent=4)
    with open(_dir + '\\' + _name + '_params.json', 'w') as outfile:
        outfile.write(json_object)
    print ('model + params saved!')

loss_log = []
progress = 0

# * begin training 
for _ in tqdm(range(_EPOCHS_)):
    if _SKIP_TRAIN_:
        print ('skipping training')
        break
    with torch.no_grad():
        # * sample batch from pool
        i = len(loss_log)
        batch_idxs = np.random.choice(_POOL_SIZE_, _BATCH_SIZE_, replace=False)
        x = pool[batch_idxs]
        
        # * re-order batch based on loss
        loss_ranks = torch.argsort(loss_func[_LOSS_FUNC_](x, target_batch, _dims=[-2, -3, -1]), descending=True)
        x = x[loss_ranks]
        
        # * re-add seed into batch
        seed_swap_rate = 1 if i < 4000 else 5
        if i % seed_swap_rate == 0:
            x[:1] = seed
            
        # * damage lowest loss in batch
        if i % _DAMG_RATE_ == 0:
            radius = random.uniform(_SIZE_*0.05, _SIZE_*0.2)
            u = random.uniform(0, 1) * _SIZE_
            v = random.uniform(0, 1) * _SIZE_
            mask = circle_mask(_SIZE_, radius, [u, v])
            x[-_NUM_DAMG_:] *= torch.tensor(mask).to(_DEVICE_)
            
    # * different loss values
    overflow_loss = 0.0
    diff_loss = 0.0
    target_loss = 0.0
    
    # * save batch before
    if i % _INFO_RATE_ == 0:
        before = x.detach().cpu()
    
    # * forward pass
    num_steps = np.random.randint(64, 96)
    for _ in range(num_steps):
        prev_x = x
        x = model(x)
        diff_loss += (x - prev_x).abs().mean()
        overflow_loss += (x - x.clamp(-2.0, 2.0))[:, :_SCALAR_CHANNEL_].square().sum()
    
    # * calculate losses
    target_loss += loss_func[_LOSS_FUNC_](x, target_batch)
    target_loss /= 2.0
    diff_loss *= 10.0
    loss = target_loss + overflow_loss + diff_loss
    
    # * backward pass
    with torch.no_grad():
        loss.backward()
        # * normalize gradients 
        for p in model.parameters():
            p.grad /= (p.grad.norm()+1e-8) 
        opt.step()
        opt.zero_grad()
        lr_sched.step()
        # * re-add batch to pool
        pool[batch_idxs] = x
        loss_log.append(loss.item())
        
        # * print out info
        if i % _INFO_RATE_ == 0:
            # * show loss plot
            clear_output(True)
            pl.plot(loss_log, '.', alpha=0.5)
            pl.yscale('log')
            pl.ylim(np.min(loss_log), loss_log[0])
            pl.show()
            
            # * show batch
            after = x.detach().cpu()
            show_batch(_BATCH_SIZE_, before, after)
            
            # * print info
            print('\rstep:', i, '\tloss:', loss.item(), '\tlr:', lr_sched.get_last_lr()[0], end='')
                
        # * save checkpoint
        if i % _SAVE_RATE_ == 0 and i != 0:
            save_model('checkpoints', model, _NAME_+'_cp'+str(i))
            
# * save final model
if not _SKIP_TRAIN_:
    save_model('models', model, _NAME_)

Play Model!

In [ ]:
import pygame
import datetime
import json

_PLAY_MODEL_ = 'models\\laplacian_cowboy'
_PLAY_DEVICE_ = 'cpu'
_RADIUS_ = 8
_SCALE_WINDOW_ = 10

# * set current device
_DEVICE_ = _PLAY_DEVICE_

params = {}
with open(_PLAY_MODEL_+'_params.json', 'r') as openfile:
    # Reading from json file
    params = json.load(openfile)
    
model = ISO_NCA(_device=_PLAY_DEVICE_)
model.load_state_dict(torch.load(_PLAY_MODEL_+'.pt', map_location=_PLAY_DEVICE_))
model.eval()

seed_img = load_image_as_tensor('..\\_seeds\\'+params['_SEED_FILE_'], params['_SIZE_'])
tensor = torch.cat([seed_img, torch.zeros([1, params['_CHANNELS_']-4, params['_SIZE_'], params['_SIZE_']])], 1).to(_PLAY_DEVICE_)

# * misc params
c = 0
angle = 0.0
fps = 0
prev_time = datetime.datetime.now()

# * start pygame
pygame.init()
pygame.display.set_caption('nca play - '+_PLAY_MODEL_)

# * model dependent params
size = params['_SIZE_']
window_size = size * _SCALE_WINDOW_
window = pygame.display.set_mode((window_size, window_size))

# * text renders
font_size = 24
font_color = (255, 255, 255)
my_font = pygame.font.SysFont('consolas', font_size)
model_surface = my_font.render('model: ' + _PLAY_MODEL_, False, font_color)
text_surface = my_font.render('angle: ' + str(angle) + 'π', False, font_color)
fps_surface = my_font.render('fps: ' + str(int(fps)), False, font_color)

# * start infinite game loop
running = True
mouse_down = False
while running:
    # empty cache
    torch.cuda.empty_cache()
    # handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                tensor = torch.cat([seed_img, torch.zeros([1, params['_CHANNELS_']-4, params['_SIZE_'], params['_SIZE_']])], 1).to(_PLAY_DEVICE_)
            # if event.key == pygame.K_UP:
            #     curr += 1
            #     if curr >= len(model_list):
            #         curr = 0
            # if event.key == pygame.K_DOWN:
            #     curr -= 1
            #     if curr < 0:
            #         curr = len(model_list)-1
            # if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
            #     # load new model
            #     model, tensor, params = load_model(model_list[curr], args.models, device)
            #     p = params['pad']
            #     pygame.display.set_caption('nca play - ' + model_list[curr])
            #     size = params['size'] + (2 * p)
            #     scale = args.scale
            #     window_size = size * scale
            #     window = pygame.display.set_mode((window_size, window_size))
            #     model_surface = my_font.render('model: ' + model_list[curr], False, font_color)
        if event.type == pygame.MOUSEWHEEL:
            angle = np.round((event.y * 0.1) + angle, decimals=1)
            text_surface = my_font.render('angle: ' + str(angle) + 'π', False, font_color)
        if event.type == pygame.MOUSEBUTTONDOWN:
            mouse_down = True
            if pygame.mouse.get_pressed(3)[2]:
                mouse = np.array(pygame.mouse.get_pos(), dtype=float)
                pos = mouse / window_size
                pos = pos * size
                dot = np.zeros_like(tensor.detach().cpu().numpy())
                dot[:, 3:, int(pos[1]), int(pos[0])] = 1.0
                tensor += torch.tensor(dot).to(_PLAY_DEVICE_)
        if event.type == pygame.MOUSEBUTTONUP:
            mouse_down = False
        if (event.type == pygame.MOUSEMOTION or event.type == pygame.MOUSEBUTTONDOWN) and mouse_down:
            mouse = np.array(pygame.mouse.get_pos(), dtype=float)
            pos = mouse / window_size
            if pygame.mouse.get_pressed(3)[0]:
                mask = circle_mask(size, _RADIUS_, pos)
                tensor[0:] *= torch.tensor(mask).to(_PLAY_DEVICE_)
            
    # * update tensor
    with torch.no_grad():
        tensor = model(tensor)
    
    # * draw tensor to window
    window.fill((255, 255, 255))
    img = to_rgb(tensor, _alpha='BLACK').squeeze()
    vis = (np.array(img.cpu())*255).astype(np.uint8)
    pixel = pygame.Surface((_SCALE_WINDOW_, _SCALE_WINDOW_))
    for j in range(size):
        for i in range(size):
            color = vis[:, i, j]
            pixel.fill(color)
            draw_me = pygame.Rect(j*_SCALE_WINDOW_, i*_SCALE_WINDOW_, _SCALE_WINDOW_, _SCALE_WINDOW_)
            window.blit(pixel, draw_me)
    
    # * calculate fps
    now = datetime.datetime.now()
    if (now - prev_time).seconds >= 1.0:
        prev_time = now
        fps_surface = my_font.render('fps: ' + str(int(fps)), False, font_color)
        fps = 0
    else:
        fps += 1       
    
    # * render text
    window.blit(model_surface, (0, 0))
    window.blit(text_surface, (window_size-font_size-150, window_size-font_size))
    window.blit(fps_surface, (0, window_size-font_size))
    
    # * flip it!
    pygame.display.flip()

# * quit it!
pygame.quit()